In [2]:
import pandas as pd
import numpy as np
import torch 

In [16]:
hartwig = pd.read_csv('newyork1924.csv')

In [18]:
hartwig

,Game Number,Move Number,Player,Move,Comment
0,1,1,White,b2b4,"WHAT?!?! In the true hypermodern style, Tartak..."
1,1,2,Black,e7e6,Maroczy (Black) makes an illogical move of his...
2,1,3,White,c1b2,Tartakower doesn't need to defend the pawn yet...
3,1,4,Black,g8f6,"Maroczy follows more conventional lines, devel..."
4,1,5,White,b4b5,Tartakower inhibits the development of Black's...
...,...,...,...,...,...
10512,110,96,Black,d7d8,The King goes to the only square that still al...
10513,110,97,White,a7a8,"To draw, all Marshall needs to do now is keep ..."
10514,110,98,Black,d8c7,Despite his material advantage and threat on t...
10515,110,99,White,a8a7,Marshall checks again...


In [17]:
num_nans = hartwig['Comment'].isna().sum()
print(num_nans)


5076


In [ ]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel

# Define model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [22]:


# Hyperparameters
learning_rate = 2e-5
epochs = 5

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(epochs):
  for board, move, commentary in train_data:
    # Preprocess board, move, and commentary
    encoded_board = tokenizer(board, return_tensors="pt")
    encoded_move = torch.tensor([move]).unsqueeze(0)

    # Encode board and move
    with torch.no_grad():
      encoder_outputs = model.bert.encoder(encoded_board)

    # Get last hidden state from encoder (encoded representation)
    encoded_representation = encoder_outputs.last_hidden_state

    # Decoder input (start token)
    decoder_input = torch.tensor([tokenizer.encode("<s>")], dtype=torch.long)

    # Generate commentary one word at a time
    for comment_word in commentary:
      # Get decoder outputs and logits
      decoder_outputs = model.gpt2(decoder_input, encoder_outputs=encoded_representation)
      logits = decoder_outputs.logits[:, -1, :]

      # Sample next word based on logits
      next_word = torch.argmax(logits, dim=-1).unsqueeze(0)
      decoder_input = torch.cat((decoder_input, next_word), dim=-1)

    # Calculate loss
    labels = torch.tensor(tokenizer.encode(commentary), dtype=torch.long)[1:]  # Ignore start token
    loss = torch.nn.functional.cross_entropy(logits.view(-1, model.vocab_size), labels)

    # Backpropagate and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


566